In [ ]:
import torch
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob


# device = torch.device("cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
device = torch.device("cuda")

#dirs = sorted(os.listdir("outs"))
#list(enumerate(dirs))

In [ ]:

ckpoint_path = "/data02/gkim/stem_cell_jwshin/outs/230811+230502_SEC1H5_wider_v3_allh_noJAX_onRA_fishdeep1_b008_in_lr0.001/epoch[00154]_tr[0.969]_va[1.000]_te[0.977]_trW[0.969]_vaW[1.000]_teW[0.977].pth.tar"

In [ ]:
#from models.fishnet2 import fishdw2, fishdw, fish150
#net = fishdw(num_classes=2, norm="bn", act="lrelu")
from models.fishnet2_2d import fishdeep1_2d
net = fishdeep1_2d(input_c=12, num_classes=2, norm="in", act="lrelu")


net = torch.nn.DataParallel(net)
state = torch.load(ckpoint_path)["network"]
net.load_state_dict(state)
model = net.module
model.to(device)
print("Net Load")

In [ ]:
net = net.module # for unwrapping from torch.dataparallel

In [ ]:
from visualizes.GradCam2d import GradCam2d
#from visualizes.GradCam import GradCam

from visualizes.GuidedBackpropReLUModel import GuidedBackpropReLUModel
#gb_model = GuidedBackpropReLUModel(model, device=device)

In [ ]:
# from datas.preprocess3d import TRAIN_AUGS_3D, TEST_AUGS_3D
# from datas.preprocess2d import TRAIN_AUGS_2D, TEST_AUGS_2D
from datas.preprocess25d import TRAIN_AUGS_25D, TEST_AUGS_25D
#from datas.BacLoader import bacLoader
#data_path = "/data2/DW/180930_bac/valid40/"
#test_loader = bacLoader(data_path + "test", 1, task="bac", sampler=False,
#                        transform=TEST_AUGS_3D, aug_rate=0,
#                        num_workers=16, shuffle=False, drop_last=False)
from datas.TomoLoader import TomoLoader
data_path = "/data02/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/"
test_loader = TomoLoader(data_path + "test", 1,
               transform=TEST_AUGS_25D, aug_rate=0,
               num_workers=16, shuffle=False, drop_last=False)

In [ ]:
# name_part = "head_layer" # "body_layer" # "tail_layer"
# idx_layer = 3
# model._modules[name_part][idx_layer]._modules['layer']._modules['layer'][0]

In [ ]:
import torch.nn.functional as F
name_part = "head_layer" # "body_layer" # "tail_layer"
idx_layer = 1
# model._modules[name_part][idx_layer]._modules['layer'][0]

In [ ]:

for i, (input_, label_, path) in enumerate(test_loader):

    torch.cuda.empty_cache()
    grad_cam = GradCam2d(model, name_part, idx_layer, device=device) #hooks=["features", "transition1"]
    gb_model = GuidedBackpropReLUModel(model=model, device=device)

    input_.requires_grad_(True)    
    
    respond_cam0, gdcam0, gdcampp0, predict = grad_cam.multicam(input_, 0)#gunhocam0, 
    respond_cam1, gdcam1, gdcampp1, predict = grad_cam.multicam(input_, 1)#gunhocam1, 
    # respond_cam2, gdcam2, gdcampp2, predict = grad_cam.multicam(input_, 2)#gunhocam1, 
    gb_img0 = gb_model(input_, 0)
    gb_img1 = gb_model(input_, 1)
    img = input_[0].detach().numpy()
    
    label = test_loader.dataset.idx_to_class[label_.item()]
    pred = test_loader.dataset.idx_to_class[predict.item()]
    fig = plt.figure(figsize=(20, 20))
    plt.suptitle("File : %s\n Label : %s, pred : %s"%(path, label, pred),
                 fontsize=16, y=0.61)
    # print(input_.shape)
    # print(img.shape)
    # print(gb_img0.shape)
    # print(np.max(gb_img0, axis = 0).shape)
    # print(np.squeeze(np.max(gb_img0, axis = 0)).shape)
    ## for 2d
    ax = plt.subplot(2, 4, 1)
    ax.set_title("RI")
    plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray")

    ax = plt.subplot(2, 4, 2)
    ax.set_title("GradCAM 0")
    plt.imshow(gdcam0, cmap = "jet")#gdcam[:, :, zdim],       cmap=cmap)
    
    ax = plt.subplot(2, 4, 3)
    ax.set_title("GuidedBP 0")
    plt.imshow(np.squeeze(np.max(gb_img0, axis = 0)), cmap = "gray")#gdcam[:, :, zdim],       cmap=cmap)
   
    ax = plt.subplot(2, 4, 4)
    ax.set_title("Multiplied 0")
    plt.imshow(np.multiply(np.squeeze(np.max(gb_img0, axis = 0)),np.resize(gdcam0, gb_img0.shape[1:])), cmap = "jet")
    
    ax = plt.subplot(2, 4, 6)
    ax.set_title("GradCAM 1")
    plt.imshow(gdcam1, cmap = "jet")#gdcam[:, :, zdim],       cmap=cmap)
    
    ax = plt.subplot(2, 4, 7)
    ax.set_title("GuidedBP 1")
    plt.imshow(np.squeeze(np.max(gb_img1, axis = 0)), cmap = "gray")#gdcam[:, :, zdim],       cmap=cmap)
   
    ax = plt.subplot(2, 4, 8)
    ax.set_title("Multiplied 1")
    plt.imshow(np.multiply(np.squeeze(np.max(gb_img1, axis = 0)),np.resize(gdcam1, gb_img1.shape[1:])), cmap = "jet")
    

    
    save_path = "/data02/gkim/stem_cell_jwshin/outs/230811+230502_SEC1H5_wider_v3_allh_noJAX_onRA_fishdeep1_b008_in_lr0.001/cams/epoch[00154]/" + name_part + "{}".format(idx_layer) +  "/" + ("True/" if label == pred else "False/")
    img_path = "/data02/gkim/stem_cell_jwshin/outs/230811+230502_SEC1H5_wider_v3_allh_noJAX_onRA_fishdeep1_b008_in_lr0.001/cams_png/epoch[00154]/" + name_part + "{}".format(idx_layer) +  "/" + ("True/" if label == pred else "False/")

    # save_path = "/data02/gkim/stem_cell_jwshin/outs/230407_MIP_b004_in_lr0.001/epoch[00009]_cams/" + name_part + "{}".format(idx_layer) +  "/" + ("True/" if label == pred else "False/")
    #"/data02/gkim/stem_cell_jwshin/outs/220803_3D_b016_lr0.001/epoch[00012]_cams_" + name_part + "{}".format(idx_layer) +  "/" + ("True/" if label == pred else "False/")
    #print(save_path + file_name)
    file_name = path[0][path[0].find("/test/") + 6:-4] + ".png"
    os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
    os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
    plt.savefig(img_path + file_name, dpi=fig.dpi)
    
    file_name = path[0][path[0].find("/test/") +  6:-4]
    dict_data = {'ri':img, 'respcam0':respond_cam0, 'gradcam0':gdcam0, 'gradcampp0':gdcampp0, 
    'respcam1':respond_cam1, 'gradcam1':gdcam1, 'gradcampp1':gdcampp1,
    'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
    'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
    savemat(save_path+file_name, dict_data)

    print(save_path + file_name)
    print(img_path + file_name)
    #print(save_path + file_name)
    
    # plt.show()
    plt.close()
    
    del grad_cam